# Life Expectancy Data Cleaning

In [36]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

Load Raw Data

In [37]:
# Load Data
df = pd.read_csv("LifeExpectancyRaw.csv")

Drop Rows

In [38]:
# Drop Rows where Life Expectancy is Missing (10 countries where the country only has one year of data)
df = df.dropna(subset=['Life expectancy '])

# Drop South Sudan Due to Large Portion of Missing Data
df = df[df['Country'] != 'South Sudan']

Convert Columns to Binary

In [39]:
# Convert Status Column to Binary
df['Status'].replace({'Developed': 1, 'Developing': 0}, inplace=True)

Manually Insert Data

In [40]:
# Manually insert population
# Load Data
pop_df = pd.read_csv("population-and-demography.csv")
pop_df = pop_df[pop_df['Entity'].isin(['Antigua and Barbuda', 'Bahamas', 'Bahrain', 'Barbados',
                 'Bolivia', 'Brunei', 'Congo','Cuba', 'Czechia', "Cote d'Ivoire",
                 "North Korea", 'Democratic Republic of Congo', 'Egypt', 'Eritrea',
                 'Gambia', 'Grenada', 'Iran', 'Kuwait', 'Kyrgyzstan', "Laos", 'Libya',
                 'Micronesia (country)', 'New Zealand', 'Oman', 'Qatar', 'South Korea',
                 'Moldova', 'Saint Lucia', 'Saint Vincent and the Grenadines',
                 'Saudi Arabia', 'Singapore', 'Slovakia', 'Somalia', 'North Macedonia',
                 'United Arab Emirates', 'United Kingdom', 'Tanzania', 'United States',
                 'Venezuela', 'Vietnam', 'Yemen'])]

pop_df['Entity'] = pop_df['Entity'].replace({
    'Bolivia': 'Bolivia (Plurinational State of)',
    'Brunei': 'Brunei Darussalam',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'North Korea': "Democratic People's Republic of Korea",
    'Iran': 'Iran (Islamic Republic of)',
    'South Korea': 'Republic of Korea',
    'Laos': "Lao People's Democratic Republic",
    'Micronesia (country)': 'Micronesia (Federated States of)',
    'Moldova': 'Republic of Moldova',
    'North Macedonia': 'The former Yugoslav republic of Macedonia',
    'Tanzania': 'United Republic of Tanzania',
    'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
    'United States': 'United States of America',
    'Venezuela': 'Venezuela (Bolivarian Republic of)',
    'Vietnam': 'Viet Nam'
})

pop_df = pop_df.rename(columns={
    'Entity': 'Country',
    "Population - Sex: all - Age: all - Variant: estimates": "Population"})

# Merge df with pop_df on Country and Year
df = df.merge(pop_df[['Country', 'Year', 'Population']], on=['Country', 'Year'], how='left', suffixes=('', '_pop_df'))

# Fill missing values in df's Population column with values from pop_df
df['Population'] = df['Population'].fillna(df['Population_pop_df'])

# Drop the additional Population column from pop_df after filling in missing values
df = df.drop(columns=['Population_pop_df'])

In [41]:
# Load measles data
measles_df = pd.read_csv('MeaslesCoverage.csv', encoding='latin1')

# Replace original values with NaN
df['Measles '] = np.nan

# Merge df with measles_df on Country and Year
measles_df = measles_df.rename(columns={'NAME': 'Country', 'YEAR': 'Year'})
merged_df = df.merge(measles_df[['Country', 'Year', 'COVERAGE']], on=['Country', 'Year'], how='left')

# Fill NaN values in the 'Measles' column with corresponding 'COVERAGE' values from measles_df
df['Measles '] = merged_df['Measles '].fillna(merged_df['COVERAGE'])

# Replace values in 'Measles' column that fall outside the expected percentage range with NaN
df.loc[df['Measles '] > 100, 'Measles '] = np.nan

In [42]:
# Manually insert child mortality
# Load Data
mortality_df = pd.read_csv("child-mortality.csv")

# Rename values and columns for merging
mortality_df['Entity'] = mortality_df['Entity'].replace({
    'Bolivia': 'Bolivia (Plurinational State of)',
    'Brunei': 'Brunei Darussalam',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Cape Verde': 'Cabo Verde',
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'North Korea': "Democratic People's Republic of Korea",
    'Iran': 'Iran (Islamic Republic of)',
    'South Korea': 'Republic of Korea',
    'Laos': "Lao People's Democratic Republic",
    'Micronesia (country)': 'Micronesia (Federated States of)',
    'Moldova': 'Republic of Moldova',
    'North Macedonia': 'The former Yugoslav republic of Macedonia',
    'Tanzania': 'United Republic of Tanzania',
    'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
    'United States': 'United States of America',
    'Venezuela': 'Venezuela (Bolivarian Republic of)',
    'Vietnam': 'Viet Nam', 
    'Russia': 'Russian Federation',
    'Eswatini': 'Swaziland', 
    'Syria': 'Syrian Arab Republic',
    'East Timor': 'Timor-Leste'
})

mortality_df = mortality_df.rename(columns={
    'Entity': 'Country',
    'Under-five mortality rate': 'under-five deaths '})

# Merge df with pop_df on Country and Year
df = df.merge(mortality_df[['Country', 'Year', 'under-five deaths ']], on=['Country', 'Year'], how='left', suffixes=('', '_mortality_df'))

# Replace under-five deaths with values from mortality_df
df['under-five deaths '] = df['under-five deaths '].astype(float) # Make columns the same dtype to avoid errors
df['under-five deaths '] = df['under-five deaths _mortality_df']

# Drop the additional column from mortality_df after filling in values
df = df.drop(columns=['under-five deaths _mortality_df'])

In [43]:
# Manually insert GDP data
gdp_df = pd.read_csv("GDP.csv", skiprows=4)
gdp_df.drop(columns=['Country Code', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', 'Unnamed: 68'], inplace=True)            # drop unnecessary columns

# Rename column names to match our data
gdp_df['Country Name'] = gdp_df['Country Name'].replace({
    'Bahamas, The': 'Bahamas',
    'Bolivia': 'Bolivia (Plurinational State of)',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'Congo, Dem. Rep.':'Democratic Republic of the Congo',
    'Congo, Rep.':'Congo',
    'Egypt, Arab Rep.':'Egypt',
    'Gambia, The': "Gambia",
    'Iran, Islamic Rep.': 'Iran (Islamic Republic of)',
    'Korea, Dem. People\'s Rep.':"Democratic People's Republic of Korea",
    'Korea, Rep.': 'Republic of Korea',
    'Kyrgyz Republic': 'Kyrgyzstan',
    "Lao PDR": "Lao People's Democratic Republic",
    'Micronesia, Fed. Sts.': 'Micronesia (Federated States of)',
    'Moldova': 'Republic of Moldova',
    'North Macedonia': 'The former Yugoslav republic of Macedonia',
    'Slovak Republic': 'Slovakia',
    'St. Lucia': 'Saint Lucia',
    'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
    'Tanzania': 'United Republic of Tanzania',
    'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
    'United States': 'United States of America',
    'Venezuela, RB': 'Venezuela (Bolivarian Republic of)',
    'Vietnam': 'Viet Nam',
    'Yemen, Rep.': 'Yemen'
})
# Turn year columns into seperate rows
gdp_df = pd.melt(gdp_df, 
                   id_vars=['Country Name', 'Indicator Name'],
                   var_name='Year', 
                   value_name='GDP')

# Change year to int to be able to merge
gdp_df['Year'] = gdp_df['Year'].astype(int)

# Merge df with gdp_df on Country and Year
gdp_df_merged = df.merge(gdp_df[['Country Name', 'Year', 'GDP']], left_on=['Country', 'Year'], 
                         right_on=['Country Name', 'Year'], 
                         how='left', 
                         suffixes=('', '_new'))

# Add Venezuela data
years = list(range(2000,2016))
gdp_ven = [11.9e3, 12.1e3, 10.8e3, 9.83e3, 11.4e3, 12.4e3, 13.4e3, 14.4e3, 14.9e3, 14.2e3, 13.8e3, 14.2e3, 14.7e3, 14.7e3, 1.4e4, 14.1e3]
venezuela_gdp = pd.DataFrame({
    'Country': 'Venezuela (Bolivarian Republic of)',
    'Year': years,
    'GDP_v': gdp_ven
})

# Add North Korea data
gdp_nk = [10.61e9, 11.02e9, 10.91e9, 11.05e9, 11.17e9, 13.03e9, 13.76e9, 14.37e9, 13.34e9, 12.04e9, 13.95e9, 15.69e9, 15.91e9, 16.57e9, 17.4e9, 16.28e9]
nk_gdp = pd.DataFrame({
    'Country': "Democratic People's Republic of Korea",
    'Year':years,
    'GDP_nk':gdp_nk
})

# Merge GDP data back into the main DataFrame on Country and Year
gdp_df_merged = gdp_df_merged.merge(venezuela_gdp, on=['Country', 'Year'], how='outer')
gdp_df_merged = gdp_df_merged.merge(nk_gdp, on=['Country', 'Year'], how='outer')

# Fill missing GDP values
gdp_df_merged['GDP'] = gdp_df_merged['GDP'].fillna(gdp_df_merged['GDP_new'])
gdp_df_merged['GDP'] = gdp_df_merged['GDP'].fillna(gdp_df_merged['GDP_v'])
gdp_df_merged['GDP'] = gdp_df_merged['GDP'].fillna(gdp_df_merged['GDP_nk']/gdp_df_merged['Population'])

# Drop joined columns after filling in missing values
df = gdp_df_merged.drop(columns=['Country Name', 'GDP_v', 'GDP_new', 'GDP_nk'])

Forward/Backward Fill and Linear Interpolation by Country

In [44]:
def fill_missing_values(group, col):
    # Interpolate values for years 2001-2014
    group.loc[(group['Year'] > 2000) & (group['Year'] < 2015), col] = group[col].interpolate(method='linear')

    # For year 2000, fill with the next year's value if available
    group.loc[group['Year'] == 2000, col] = group.loc[group['Year'] == 2000, col].fillna(group.loc[group['Year'] == 2001, col])

    # For year 2015, fill with the previous year's value if available
    group.loc[group['Year'] == 2015, col] = group.loc[group['Year'] == 2015, col].fillna(group.loc[group['Year'] == 2014, col])

    # Optionally use forward fill and backward fill for remaining NaNs
    group[col] = group[col].fillna(method='ffill').fillna(method='bfill')

    return group

start_col = df.columns.get_loc('Status')
end_col = df.columns.get_loc('Schooling')
for col in df.columns[start_col:end_col + 1]:
  df = df.groupby('Country', group_keys=False).apply(lambda group: fill_missing_values(group, col))
  df = df.reset_index(drop=True)

Remove Values that Fall Outside the Reasonable Range

In [45]:
# Replace values outside the range with NaN
df.loc[~df['Adult Mortality'].between(1, 600), 'Adult Mortality'] = np.nan

KNN

In [46]:
# Store the 'Country' column temporarily in order to perform KNN on Numeric Data
country_col = df['Country']
df = df.drop(columns=['Country'])

# Apply KNN Imputer for Remaining NaN Values
knn_imputer = KNNImputer(n_neighbors=2)
df = pd.DataFrame(knn_imputer.fit_transform(df), columns=df.columns)

# Re-add 'Country' to the DataFrame
df['Country'] = country_col.values

Formatting

In [47]:
# strip columns of leading and trailing whitespace
df.columns = df.columns.str.strip()

# rename columns to create consistent capitalization
df.rename(columns={'Life expectancy':'Life Expectancy', 'infant deaths':'Infant Deaths', 'percentage expenditure':'Percentage Expenditure', 
                   'under-five deaths':'Under-five Deaths', 'Total expenditure':'Total Expenditure', 'thinness  1-19 years':'Thinness 1-19 Years', 
                   'thinness 5-9 years':'Thinness 5-9 Years', 'Income composition of resources':'Income Composition of Resources'}, inplace=True)

Assertions

In [48]:
assert df.duplicated().sum() == 0, "The DataFrame contains duplicate rows."
assert df.isnull().sum().sum() == 0, "There are null values in the DataFrame."
assert df.groupby('Country')['Year'].apply(lambda x: set(range(2000, 2016)).issubset(x)).all(), "Not all countries have entries for each year from 2000 to 2015."
assert df['Life Expectancy'].between(35, 100).all(), "Life expectancy values are out of the specified range"
assert df['Adult Mortality'].between(1, 600).all(), "Adult mortality values are out of the specified range"
# assert df['Infant Deaths'].between(1, 600).all(), "Infant mortality values are out of the specified range"
assert df['Alcohol'].between(0, 18).all(), "Alcohol consumption values are out of the specified range"
assert df['Percentage Expenditure'].between(0, 30000).all(), "Percentage expenditure values are out of the specified range"
assert df['Hepatitis B'].between(0, 100).all(), "Hepatitis B values are out of the specified range"
assert df['Measles'].between(0, 100).all(), "Measles values are out of the specified range"
# assert df['BMI'].between(10, 40).all(), "BMI values are out of the specified range"
# assert df['Under-five Deaths'].between(1, 1000).all(), "Under-five mortality values are out of the specified range"
assert df['Polio'].between(0, 100).all(), "Polio values are out of the specified range"
assert df['Total Expenditure'].between(0, 30000).all(), "Total expenditure values are out of the specified range"
assert df['Diphtheria'].between(0, 100).all(), "Diptheria values are out of the specified range"
assert df['HIV/AIDS'].between(0, 900).all(), "HIV/AIDS values are out of the specified range"
assert df['GDP'].between(0, 300000).all(), "GDP values are out of the specified range"
# assert df['Population'].between(100, 2000000000).all(), "Population values are out of the specified range"
assert df['Thinness 1-19 Years'].between(0, 60).all(), "Thinness (1-19) values are out of the specified range"
assert df['Thinness 5-9 Years'].between(0, 60).all(), "Thinness (5-9) values are out of the specified range"
assert df['Income Composition of Resources'].between(0, 1).all(), "Income composition resources values are out of the specified range"
assert df['Schooling'].between(0, 25).all(), "Schooling values are out of the specified range"

Export Clean Data to CSV

In [49]:
df.to_csv('LifeExpectancyClean.csv')